<a href="https://colab.research.google.com/github/yungshan629/NTU-Discrete-Mathematics/blob/main/%E9%9B%A2%E6%95%A3%E6%95%B8%E5%AD%B8%E4%BD%9C%E6%A5%AD2_20241118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 定義海戰的基本資料

為了讓作業能夠順利進行，我將程式碼分成4個部分：第一部分是遊戲的基本設定，包括網格系統、移動系統、艦隊系統、飛彈系統和主遊戲邏輯等等。由於程式碼很長也很亂，因此我會多寫一些註釋，方便和作業要求對照。第二部分是作業的重點，即攻、守的決策矩陣。使用的參數都是在跑過敏感度分析後，最適合的參數。第三部分是模擬及各種統計分析包括存活率、擊沉率等等。第四部分亦是作業的重點之一，包括敏感度分析和賽局分析。

1. 基礎系統設計
* 船艦種類與屬性設定

\begin{array}{|l|c|c|c|c|}
\hline
\text{選項/準則} & \text{血條(沉沒條件)} & \text{音速飛彈} & \text{超音速飛彈} & \text{移動格數} \\
\hline
\text{大型船艦} & \text{3} & \text{5} & \text{4} & \text{4} \\
\hline
\text{中型船艦} & \text{2} & \text{4} & \text{2} & \text{3} \\
\hline
\text{小型船艦} & \text{1} & \text{0} & \text{3} & \text{2} \\
\hline
\end{array}

* 核心系統
 * 網格系統：處理和網格相關事務，包括計算、驗證位置
 * 移動系統：處理移動相關計算
 * 飛彈系統：定義飛彈攻擊範圍
 * 艦隊系統：管理雙方艦隊配置，統計艦隊狀態
 * 戰鬥記錄系統：記錄戰鬥過程，提供戰鬥數據分析

2. 遊戲規則
3. 統計與分析功能



In [ ]:
import numpy as np
from enum import Enum
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import random
import math


class ShipType(Enum):
    LARGE = "large"
    MEDIUM = "medium"
    SMALL = "small"


class MissileType(Enum):
    SONIC = "sonic"
    HYPERSONIC = "hypersonic"


@dataclass
class Ship:
    type: ShipType
    position: Tuple[int, int]
    health: int
    sonic_missiles: int
    hypersonic_missiles: int
    speed: int
    name: str
    hit_count: int = 0
    has_appeared: bool = False
    current_state: str = "hidden"

    @classmethod
    # 船艦基本定義, 如表格所示
    def create(cls, ship_type: ShipType, position: Tuple[int, int], name: str):
        if ship_type == ShipType.LARGE:
            return cls(ship_type, position, 3, 5, 4, 4, f"{name}_Large")
        elif ship_type == ShipType.MEDIUM:
            return cls(ship_type, position, 2, 4, 2, 3, f"{name}_Medium")
        else:
            return cls(ship_type, position, 1, 0, 3, 2, f"{name}_Small")

    def is_sunk(self) -> bool:
        if self.hit_count >= self.health:
            self.current_state = "sunk"
            return True
        return False

    def get_status(self) -> Dict:
        return {
            'name': self.name,
            'type': self.type.value,
            'position': self.position,
            'health': self.health - self.hit_count,
            'sonic_missiles': self.sonic_missiles,
            'hypersonic_missiles': self.hypersonic_missiles,
            'state': self.current_state,
            'has_appeared': self.has_appeared
        }


class MovementSystem:
    def __init__(self, grid_size: int):
        self.grid_size = grid_size

    def move_ship(self, ship: Ship, direction: str) -> Tuple[int, int]:
        if not ship.has_appeared or ship.is_sunk():
            return ship.position
        x, y = ship.position
        new_x, new_y = x, y
        if direction == 'up':
            new_y = max(0, y - ship.speed)
        elif direction == 'down':
            new_y = min(self.grid_size - 1, y + ship.speed)
        elif direction == 'left':
            new_x = max(0, x - ship.speed)
        elif direction == 'right':
            new_x = min(self.grid_size - 1, x + ship.speed)
        return (new_x, new_y)

    def validate_movement(self, ship: Ship, new_position: Tuple[int, int],
                          grid: List[List[Optional[Ship]]]) -> bool:
        if not (0 <= new_position[0] < self.grid_size and
                0 <= new_position[1] < self.grid_size):
            return False

        if (grid[new_position[0]][new_position[1]] is not None and
            grid[new_position[0]][new_position[1]] != ship):
            return False
        old_x, old_y = ship.position
        new_x, new_y = new_position
        move_distance = abs(new_x - old_x) + abs(new_y - old_y)
        return move_distance <= ship.speed

    def get_valid_positions(self, ship: Ship, grid_size: int,
                            all_ships: List[Ship]) -> List[Tuple[int, int]]:

        valid_positions = []
        current_x, current_y = ship.position
        speed = ship.speed

        for dx in range(-speed, speed + 1):
            for dy in range(-speed, speed + 1):
                if abs(dx) + abs(dy) > speed:
                    continue
                new_x = current_x + dx
                new_y = current_y + dy
                if 0 <= new_x < grid_size and 0 <= new_y < grid_size:
                    position_occupied = any(
                        other.position == (new_x, new_y)
                        for other in all_ships
                        if other != ship
                    )
                    if not position_occupied:
                        valid_positions.append((new_x, new_y))

        return valid_positions


class MissileSystem:
    def __init__(self, grid_size: int):
        self.grid_size = grid_size

    def check_hit(self, attacker_pos: Tuple[int, int], target_pos: Tuple[int, int],
                 missile_type: MissileType) -> bool:
        distance = self.calculate_distance(attacker_pos, target_pos)
        if not self.is_in_range(distance, missile_type):
            print(f"目標距離 {distance} 超出射程")
            return False

        result = self.is_in_pattern(attacker_pos, target_pos, missile_type)
        if not result:
            print("目標不在攻擊範圍內")
        return result

    def calculate_distance(self, pos1: Tuple[int, int], pos2: Tuple[int, int]) -> float:
        dx = abs(pos1[0] - pos2[0])
        dy = abs(pos1[1] - pos2[1])
        return math.sqrt(dx*dx + dy*dy)

    def is_in_range(self, distance: float, missile_type: MissileType) -> bool:
        max_range = {
            MissileType.SONIC: 4,
            MissileType.HYPERSONIC: 5
        }
        return distance <= max_range[missile_type]

    def is_in_pattern(self, attacker_pos: Tuple[int, int],
                     target_pos: Tuple[int, int],
                     missile_type: MissileType) -> bool:
        ax, ay = attacker_pos
        tx, ty = target_pos

        if missile_type == MissileType.SONIC:
            return ax == tx or ay == ty
        else:
            return (abs(ax - tx) <= 1 and abs(ay - ty) <= 1)

    def calculate_covered_positions(self, attack_pos: Tuple[int, int],
                                    missile_type: MissileType) -> List[Tuple[int, int]]:
        covered = []
        x, y = attack_pos

        if missile_type == MissileType.SONIC:  # 音速飛彈, 擊中範圍為十字
            for dx in range(-4, 5):
                new_x = x + dx
                if 0 <= new_x < self.grid_size:
                  covered.append((new_x, y))
            for dy in range(-4, 5):
                new_y = y + dy
                if 0 <= new_y < self.grid_size:
                  covered.append((x, new_y))

        else:                   # 超音速飛彈, 擊中範圍為九宮格
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    new_x, new_y = x + dx, y + dy
                    if (0 <= new_x < self.grid_size and
                        0 <= new_y < self.grid_size):
                        covered.append((new_x, new_y))
        return list(set(covered))



class GridSystem:
    def __init__(self, size: int):
        self.size = size
        self.grid = [[None for _ in range(size)] for _ in range(size)]

    def place_ship(self, ship: Ship, position: Tuple[int, int]) -> bool:
        x, y = position
        if self.grid[x][y] is not None:
            return False

        if ship.has_appeared:
            old_x, old_y = ship.position
            self.grid[old_x][old_y] = None

        ship.position = position
        ship.has_appeared = True
        ship.current_state = "active"

        self.grid[x][y] = ship
        return True

    def get_empty_positions(self) -> List[Tuple[int, int]]:
        empty = []
        for x in range(self.size):
            for y in range(self.size):
                if self.grid[x][y] is None:
                    empty.append((x, y))
        random.shuffle(empty)
        return empty

    def find_strategic_position(self, appeared_ships_count: int = 0) -> Optional[Tuple[int, int]]:
        empty_positions = self.get_empty_positions()
        if not empty_positions:
            print("警告：找不到空位置")
            return None

        if appeared_ships_count < 4:
            corners = [(0, 0), (0, self.size-1),
                      (self.size-1, 0), (self.size-1, self.size-1)]
            available_corners = [pos for pos in corners if pos in empty_positions]
            if available_corners:
                return random.choice(available_corners)

        best_position = None
        max_min_distance = 0

        for pos in empty_positions:
            min_distance = self.calculate_min_distance_to_ships(pos)
            if min_distance > max_min_distance:
                max_min_distance = min_distance
                best_position = pos

        return best_position or random.choice(empty_positions)

    def calculate_min_distance_to_ships(self, position: Tuple[int, int]) -> float:
        min_distance = float('inf')
        for x in range(self.size):
            for y in range(self.size):
                if self.grid[x][y] is not None:
                    distance = math.sqrt((position[0]-x)**2 + (position[1]-y)**2)
                    min_distance = min(min_distance, distance)
        return min_distance if min_distance != float('inf') else 0

    def is_position_valid(self, position: Tuple[int, int]) -> bool:
        x, y = position
        return 0 <= x < self.size and 0 <= y < self.size

    def get_ship_at_position(self, position: Tuple[int, int]) -> Optional[Ship]:
        x, y = position
        if self.is_position_valid(position):
            return self.grid[x][y]
        return None

    def count_appeared_ships(self) -> int:
        count = 0
        for x in range(self.size):
            for y in range(self.size):
                if self.grid[x][y] is not None:
                    count += 1
        return count

    def get_all_ships_positions(self) -> List[Tuple[Tuple[int, int], Ship]]:
        ships = []
        for x in range(self.size):
            for y in range(self.size):
                if self.grid[x][y] is not None:
                    ships.append(((x, y), self.grid[x][y]))
        return ships

    def remove_ship(self, position: Tuple[int, int]) -> bool:
        x, y = position
        if self.is_position_valid(position) and self.grid[x][y] is not None:
            self.grid[x][y] = None
            return True
        return False

    def get_grid_status(self) -> List[List[Optional[Dict]]]:
        status = []
        for row in self.grid:
            status_row = []
            for cell in row:
                if cell is None:
                    status_row.append(None)
                else:
                    status_row.append(cell.get_status())
            status.append(status_row)
        return status


class FleetSystem:
    def __init__(self, grid_system: GridSystem):
        self.grid_system = grid_system
        self.team_A = self._initialize_fleet("A")
        self.team_B = self._initialize_fleet("B")
        self._initial_ship_placement()

    def _initialize_fleet(self, team_name: str) -> Dict[ShipType, List[Ship]]:
        fleet = {ShipType.LARGE: [], ShipType.MEDIUM: [], ShipType.SMALL: []}

        # 甲方: 大型*4, 中型*2, 小型*2;  鄉方: 大型*3, 中型*3, 小型*3
        ship_counts = {
            ShipType.LARGE: 4 if team_name == "A" else 3,
            ShipType.MEDIUM: 2 if team_name == "A" else 3,
            ShipType.SMALL: 2 if team_name == "A" else 3
        }

        for ship_type, count in ship_counts.items():
            for i in range(count):
                ship = Ship.create(ship_type, (0, 0), f"Team_{team_name}")
                fleet[ship_type].append(ship)

        return fleet

    def _initial_ship_placement(self):
        for team in [self.team_A, self.team_B]:
            for ship_type, ships in team.items():
                if ships:
                    ship = random.choice(ships)
                    position = self.grid_system.find_strategic_position(
                        self.count_appeared_ships(team)
                    )
                    if position:
                        self.grid_system.place_ship(ship, position)

    def get_active_ships(self, team: Dict[ShipType, List[Ship]]) -> List[Ship]:
        active_ships = []
        for ships in team.values():
            for ship in ships:
                if not ship.is_sunk() and ship.has_appeared:
                    active_ships.append(ship)
        return active_ships

    def count_ships(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, int]:
        return {
            'large': len(team[ShipType.LARGE]),
            'medium': len(team[ShipType.MEDIUM]),
            'small': len(team[ShipType.SMALL])
        }

    def count_alive_ships(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, int]:
        alive_counts = {'large': 0, 'medium': 0, 'small': 0}
        for ship_type, ships in team.items():
            for ship in ships:
                if not ship.is_sunk():
                    alive_counts[ship_type.value] += 1
        return alive_counts

    def count_appeared_ships(self, team: Dict[ShipType, List[Ship]]) -> int:
        count = 0
        for ships in team.values():
            count += sum(1 for ship in ships if ship.has_appeared)
        return count

    def force_appear_ships(self, team: Dict[ShipType, List[Ship]]) -> None:
        hidden_ships = []
        attempts = 0
        max_attempts = 100

        for ships in team.values():
            hidden_ships.extend([ship for ship in ships if not ship.has_appeared and not ship.is_sunk()])

        if hidden_ships:
            print(f"需要強制出現 {len(hidden_ships)} 艘隱藏船艦")

        while hidden_ships and attempts < max_attempts:
            for ship in hidden_ships[:]:
                position = self.grid_system.find_strategic_position(self.count_appeared_ships(team))
                if position:
                    if self.grid_system.place_ship(ship, position):
                        hidden_ships.remove(ship)
                        print(f"{ship.name} 成功出現在位置 {position}")
                attempts += 1

        if hidden_ships:
            print(f"警告：仍有 {len(hidden_ships)} 艘船艦無法找到位置出現")
            empty_positions = self.grid_system.get_empty_positions()
            for ship in hidden_ships:
                if empty_positions:
                    position = empty_positions.pop()
                    self.grid_system.place_ship(ship, position)
                    print(f"緊急處理：{ship.name} 被迫出現在位置 {position}")

    def auto_appear_ships(self, team: Dict[ShipType, List[Ship]], appearance_chance: float = 0.3) -> None:
        for ships in team.values():
            for ship in ships:
                if (not ship.has_appeared and
                    not ship.is_sunk() and
                    random.random() < appearance_chance):
                    position = self.grid_system.find_strategic_position(self.count_appeared_ships(team))
                    if position:
                        self.grid_system.place_ship(ship, position)

    def get_team_status(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, List[Dict]]:
        status = {
            "appeared": [],
            "hidden": [],
            "sunk": []
        }

        for ships in team.values():
            for ship in ships:
                ship_status = ship.get_status()
                if ship.is_sunk():
                    status["sunk"].append(ship_status)
                elif ship.has_appeared:
                    status["appeared"].append(ship_status)
                else:
                    status["hidden"].append(ship_status)

        return status

    def calculate_survival_rate(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, float]:
        survival_rates = {}
        for ship_type, ships in team.items():
            total = len(ships)
            alive = sum(1 for ship in ships if not ship.is_sunk())
            survival_rates[ship_type.value] = alive / total if total > 0 else 0
        return survival_rates


class BattleLogSystem:
    def __init__(self):
        self.battle_log = []

    def log_game_start(self, game_info: Dict):
        self.battle_log.append({
            'type': 'game_start',
            'timestamp': random.randint(1000000, 9999999),
            'info': game_info
        })

    def log_combat_skip(self, round_num: int, is_first_attack: bool, reason: str):
        self.battle_log.append({
            'type': 'combat_skip',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'reason': reason
        })

    def log_attack_move(self, round_num: int, is_first_attack: bool,
                       ship: Ship, from_pos: Tuple[int, int],
                       to_pos: Tuple[int, int], scores: Dict):

        self.battle_log.append({
            'type': 'attack_move',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'ship': ship.name,
            'from_position': from_pos,
            'to_position': to_pos,
            'scores': scores
        })

    def log_attacker_selected(self, round_num: int, is_first_attack: bool,
                            ship: Ship, position: Tuple[int, int], scores: Dict):
        self.battle_log.append({
            'type': 'attacker_selected',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'ship': ship.name,
            'position': position,
            'scores': scores
        })

    def log_target_selected(self, round_num: int, is_first_attack: bool,
                          ship: Ship, position: Tuple[int, int], scores: Dict):
        self.battle_log.append({
            'type': 'target_selected',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'ship': ship.name,
            'position': position,
            'scores': scores
        })

    def log_missile_selected(self, round_num: int, is_first_attack: bool,
                           missile_type: MissileType, scores: Dict):
        self.battle_log.append({
            'type': 'missile_selected',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'missile_type': missile_type.value,
            'scores': scores
        })

    def log_attack_result(self, round_num: int, is_first_attack: bool,
                         attacker: Ship, target: Ship, missile_type: MissileType,
                         hit_success: bool, target_sunk: bool,
                         covered_positions: List[Tuple[int, int]]):
        self.battle_log.append({
            'type': 'attack_result',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'attacker': attacker.name,
            'target': target.name,
            'missile_type': missile_type.value,
            'hit': hit_success,
            'target_sunk': target_sunk,
            'covered_positions': covered_positions
        })

    def log_attack_failed(self, round_num: int, is_first_attack: bool,
                         attacker: Ship, target: Ship, reason: str):
        self.battle_log.append({
            'type': 'attack_failed',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'attacker': attacker.name,
            'target': target.name,
            'reason': reason
        })

    def log_defense_evaluation(self, round_num: int, is_first_attack: bool,
                             ship: Ship, scores: Dict):

        self.battle_log.append({
            'type': 'defense_evaluation',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'ship': ship.name,
            'scores': scores
        })

    def log_defense_move(self, round_num: int, is_first_attack: bool,
                        ship: Ship, from_pos: Tuple[int, int],
                        to_pos: Tuple[int, int], scores: Dict):
        self.battle_log.append({
            'type': 'defense_move',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'ship': ship.name,
            'from_position': from_pos,
            'to_position': to_pos,
            'scores': scores
        })

    def log_defense_skip(self, round_num: int, is_first_attack: bool, reason: str):

        self.battle_log.append({
            'type': 'defense_skip',
            'round': round_num,
            'phase': 'first' if is_first_attack else 'second',
            'reason': reason
        })

    def get_battle_log(self) -> List[Dict]:

        return self.battle_log.copy()


class NavalBattleBase:
    def __init__(self, grid_size: int = 8):
        self.grid_system = GridSystem(grid_size)
        self.fleet_system = FleetSystem(self.grid_system)
        self.missile_system = MissileSystem(grid_size)
        self.movement_system = MovementSystem(grid_size)
        self.grid_size = grid_size
        self.current_round = 0

    def advance_round(self):
        self.current_round += 1
        if self.current_round == 3:
            self.fleet_system.force_appear_ships(self.fleet_system.team_A)
            self.fleet_system.force_appear_ships(self.fleet_system.team_B)
        else:
            self.fleet_system.auto_appear_ships(self.fleet_system.team_A)
            self.fleet_system.auto_appear_ships(self.fleet_system.team_B)

    def get_grid_status(self):
        return self.grid_system.get_grid_status()

    def get_ships_status(self):
        return {
            "Team_A": self.fleet_system.get_team_status(self.fleet_system.team_A),
            "Team_B": self.fleet_system.get_team_status(self.fleet_system.team_B)
        }

    def verify_game_rules(self) -> Dict[str, bool]:
        verifications = {
            "all_ships_appeared_by_round_3": True,
            "correct_ship_counts": True,
            "valid_positions": True
        }

        if self.current_round >= 3:
            for team in [self.fleet_system.team_A, self.fleet_system.team_B]:
                for ships in team.values():
                    for ship in ships:
                        if not ship.has_appeared and not ship.is_sunk():
                            verifications["all_ships_appeared_by_round_3"] = False


        expected_a = {ShipType.LARGE: 4, ShipType.MEDIUM: 2, ShipType.SMALL: 2}
        expected_b = {ShipType.LARGE: 3, ShipType.MEDIUM: 3, ShipType.SMALL: 3}

        actual_a = self.fleet_system.count_ships(self.fleet_system.team_A)
        actual_b = self.fleet_system.count_ships(self.fleet_system.team_B)

        if actual_a != expected_a or actual_b != expected_b:
            verifications["correct_ship_counts"] = False


        ships_positions = self.grid_system.get_all_ships_positions()
        for pos, ship in ships_positions:
            if ship.position != pos:
                verifications["valid_positions"] = False

        return verifications



# 定義決策矩陣

決策矩陣共有5個，其中3個是攻擊矩陣，2個是防守矩陣。攻擊矩陣分別是用來決定發動攻擊的船艦、攻擊對象和使用飛彈，防守矩陣則是用來決定移動那艘船艦及移動位置。

1. 攻擊矩陣：決定用那艘船發動攻擊
 * 剩餘彈藥量(40%)：剩餘彈藥越多分數越高
 * 生命值(35%)：生命值越高越適合攻擊
 * 機動性(25%)：根據船的類型評估移動能力

\begin{array}{|l|c|c|c|c|}
\hline
\text{選項/準則} & \text{剩餘彈藥量(0.4)} & \text{生命值(0.35)} & \text{機動性(0.25)} & \text{加權總分} \\
\hline
\text{大型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{中型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{小型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\end{array}

<br>

2. 攻擊矩陣：決定攻擊對象
 * 生命值(50%)：生命值越低，優先度越高
 * 威脅度(30%)：大型艦威脅度最高，小型艦威脅度最低
 * 機動性(20%)：評估目標的移動能力

\begin{array}{|l|c|c|c|c|}
\hline
\text{選項/準則} & \text{剩餘生命值(0.5)} & \text{艦種威脅(0.3)} & \text{機動性(0.2)} & \text{加權總分} \\
\hline
\text{敵方大型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{敵方中型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{敵方小型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\end{array}

<br>

3. 攻擊矩陣：決定使用飛彈
 * 彈藥量(40%)：
 * 射程評分(30%)：
 * 攻擊模式評分(30%)：


\begin{array}{|l|c|c|c|}
\hline
\text{選項/準則} & \text{剩餘數量(0.4)} & \text{射程(0.3)} & \text{攻擊模式(0.3)} & \text{加權總分} \\
\hline
\text{音速飛彈} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{超音速飛彈} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\end{array}

<br>

4. 防守矩陣：決定移動船隻
 * 威脅評分(50%)：評估受到的威脅程度
 * 生命值評分(30%)：生命值較低的優先撤離
 * 火力重要性(20%)：剩餘彈藥量的重要程度


\begin{array}{|l|c|c|c|c|}
\hline
\text{選項/準則} & \text{受威脅度(0.5)} & \text{生命值(0.3)} & \text{火力重要性(0.2)} & \text{加權總分} \\
\hline
\text{大型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{中型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{小型艦} & \text{[0-9分]} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\end{array}

<br>

5. 附守矩陣：決定移動位置
 * 脫離威脅(60%)：評估新位置的安全程度
 * 分散程度(40%)；避免船艦過於集中


\begin{array}{|l|c|c|c|}
\hline
\text{選項/準則} & \text{脫離威脅(0.6)} & \text{分散程度(0.4)} & \text{加權總分} \\
\hline
\text{位置1} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{位置2} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\text{...} & \text{[0-9分]} & \text{[0-9分]} & \text{計算結果} \\
\hline
\end{array}



In [ ]:

@dataclass
class WeightConfig:
    weights: Dict[str, float]

    def __post_init__(self):
        self.validate_weights()

    def validate_weights(self):
        total = sum(self.weights.values())
        if not math.isclose(total, 1.0, rel_tol=1e-9):
            raise ValueError(f"Weights must sum to 1.0, got {total}")

    def adjust_weight(self, key: str, new_value: float):
        if key not in self.weights:
            raise KeyError(f"Weight key {key} not found")

        old_value = self.weights[key]
        delta = new_value - old_value

        other_keys = [k for k in self.weights.keys() if k != key]
        if other_keys:
            adjustment = -delta / len(other_keys)
            for other_key in other_keys:
                self.weights[other_key] += adjustment

        self.weights[key] = new_value
        self.validate_weights()


@dataclass
class ShipScore:
    scores: Dict[str, float]
    weighted_total: float

@dataclass
class MissileScore:
    scores: Dict[str, float]
    weighted_total: float

class AttackerSelectionMatrix:
    DEFAULT_WEIGHTS = {
        "ammo": 0.4,
        "health": 0.35,
        "mobility": 0.25
    }


    def __init__(self, weight_config: Optional[WeightConfig] = None):
        self.weight_config = weight_config or WeightConfig(self.DEFAULT_WEIGHTS.copy())


    @classmethod
    def calculate_ammo_score(cls, ship: Ship) -> float:
        initial_ammo = {
            ShipType.LARGE: 9,  # 音速5 + 超音速4
            ShipType.MEDIUM: 6, # 音速4 + 超音速2
            ShipType.SMALL: 3   # 超音速3
        }
        current_ammo = ship.sonic_missiles + ship.hypersonic_missiles
        return (current_ammo / initial_ammo[ship.type]) * 9

    @classmethod
    def calculate_health_score(cls, ship: Ship) -> float:
        max_health = {ShipType.LARGE: 3, ShipType.MEDIUM: 2, ShipType.SMALL: 1}
        current_health = max_health[ship.type] - ship.hit_count
        return (current_health / max_health[ship.type]) * 9

    @classmethod
    def calculate_mobility_score(cls, ship: Ship) -> float:
        mobility_scores = {
            ShipType.LARGE: 8,  # 4格移動
            ShipType.MEDIUM: 6, # 3格移動
            ShipType.SMALL: 4   # 2格移動
        }
        return mobility_scores[ship.type]


    def evaluate(self, ship: Ship) -> ShipScore:
        scores = {
            "ammo": self.calculate_ammo_score(ship),
            "health": self.calculate_health_score(ship),
            "mobility": self.calculate_mobility_score(ship)
        }

        weighted_total = sum(
            scores[key] * self.weight_config.weights[key]
            for key in scores
        )

        return ShipScore(scores, weighted_total)

    def get_weights(self) -> Dict[str, float]:
        return self.weight_config.weights.copy()

    def set_weights(self, new_weights: Dict[str, float]):
        self.weight_config = WeightConfig(new_weights)



class TargetSelectionMatrix:
    HEALTH_WEIGHT = 0.5
    THREAT_WEIGHT = 0.3
    MOBILITY_WEIGHT = 0.2

    @classmethod
    def calculate_health_score(cls, ship: Ship) -> float:
        max_health = {ShipType.LARGE: 3, ShipType.MEDIUM: 2, ShipType.SMALL: 1}
        current_health = max_health[ship.type] - ship.hit_count
        return (1 - current_health / max_health[ship.type]) * 9

    @classmethod
    def calculate_threat_score(cls, ship: Ship) -> float:
        threat_scores = {
            ShipType.LARGE: 9,
            ShipType.MEDIUM: 6,
            ShipType.SMALL: 3
        }
        return threat_scores[ship.type]

    @classmethod
    def calculate_mobility_score(cls, ship: Ship) -> float:
        mobility_scores = {
            ShipType.LARGE: 8,
            ShipType.MEDIUM: 6,
            ShipType.SMALL: 4
        }
        return mobility_scores[ship.type]

    @classmethod
    def evaluate(cls, ship: Ship) -> ShipScore:
        scores = {
            "health": cls.calculate_health_score(ship),
            "threat": cls.calculate_threat_score(ship),
            "mobility": cls.calculate_mobility_score(ship)
        }

        weighted_total = (
            scores["health"] * cls.HEALTH_WEIGHT +
            scores["threat"] * cls.THREAT_WEIGHT +
            scores["mobility"] * cls.MOBILITY_WEIGHT
        )

        return ShipScore(scores, weighted_total)

class MissileSelectionMatrix:
    AMMO_WEIGHT = 0.4      # 彈藥權重
    RANGE_WEIGHT = 0.3     # 距離權重
    PATTERN_WEIGHT = 0.3   # 攻擊模式權重

    @classmethod
    def calculate_ammo_score(cls, ship: Ship, missile_type: MissileType) -> float:
        if missile_type == MissileType.SONIC:
            current = ship.sonic_missiles
            initial = {ShipType.LARGE: 5, ShipType.MEDIUM: 4, ShipType.SMALL: 0}[ship.type]
        else:
            current = ship.hypersonic_missiles
            initial = {ShipType.LARGE: 4, ShipType.MEDIUM: 2, ShipType.SMALL: 3}[ship.type]

        return (current / initial) * 9 if initial > 0 else 0

    @classmethod
    def calculate_range_score(cls, distance: float, missile_type: MissileType) -> float:
        max_range = {
            MissileType.SONIC: 4,      # 音速飛彈射程
            MissileType.HYPERSONIC: 5  # 超音速飛彈射程
        }[missile_type]

        # 如果超出射程，給予0分
        if distance > max_range:
            return 0

        # 根據距離比例評分
        range_ratio = distance / max_range
        if range_ratio <= 0.5:         # 近距離
            return 9
        elif range_ratio <= 0.8:       # 中距離
            return 7
        else:                          # 遠距離
            return 5



    @classmethod
    def calculate_pattern_score(cls, attacker_pos: Tuple[int, int],
                              target_pos: Tuple[int, int],
                              missile_type: MissileType) -> float:
        ax, ay = attacker_pos
        tx, ty = target_pos

        if missile_type == MissileType.SONIC:
            # 音速飛彈：十字範圍
            # 如果目標在同一行或同一列，給高分
            if ax == tx or ay == ty:
                return 9
            # 如果接近十字線，給中等分數
            elif abs(ax - tx) <= 1 or abs(ay - ty) <= 1:
                return 6
            return 3
        else:
            # 超音速飛彈：九宮格範圍
            # 在九宮格範圍內給高分
            if abs(ax - tx) <= 1 and abs(ay - ty) <= 1:
                return 9
            # 接近九宮格範圍給中等分數
            elif abs(ax - tx) <= 2 and abs(ay - ty) <= 2:
                return 6
            return 3

    @classmethod
    def evaluate(cls, ship: Ship, missile_type: MissileType,
                distance: float, target_pos: Tuple[int, int]) -> MissileScore:
        scores = {
            "ammo": cls.calculate_ammo_score(ship, missile_type),
            "range": cls.calculate_range_score(distance, missile_type),
            "pattern": cls.calculate_pattern_score(ship.position, target_pos, missile_type)
        }

        weighted_total = (
            scores["ammo"] * cls.AMMO_WEIGHT +
            scores["range"] * cls.RANGE_WEIGHT +
            scores["pattern"] * cls.PATTERN_WEIGHT
        )

        return MissileScore(scores, weighted_total)

    @classmethod
    def print_evaluation(cls, scores: MissileScore, missile_type: MissileType):
        print(f"\n=== 飛彈評估 ({missile_type.value}) ===")
        print(f"彈藥評分: {scores.scores['ammo']:.1f}")
        print(f"射程評分: {scores.scores['range']:.1f}")
        print(f"模式評分: {scores.scores['pattern']:.1f}")
        print(f"總評分: {scores.weighted_total:.1f}")




@dataclass
class DefenseShipScore:
   scores: Dict[str, float]
   weighted_total: float

@dataclass
class MovePositionScore:
   scores: Dict[str, float]
   weighted_total: float

class DefenseShipSelectionMatrix:

   THREAT_WEIGHT = 0.5
   HEALTH_WEIGHT = 0.3
   FIREPOWER_WEIGHT = 0.2

   @classmethod
   def calculate_threat_score(cls, ship: Ship, enemy_ships: List[Ship]) -> float:
       threats = sum(1 for enemy in enemy_ships
                    if cls.is_in_attack_range(ship, enemy))

       if threats >= 2:
           return 9.0
       elif threats == 1:
           return 6.0
       return 3.0

   @staticmethod
   def is_in_attack_range(ship: Ship, enemy: Ship) -> bool:
       dx = abs(ship.position[0] - enemy.position[0])
       dy = abs(ship.position[1] - enemy.position[1])
       distance = math.sqrt(dx*dx + dy*dy)
       return distance <= 5  # 假設攻擊範圍是5格

   @classmethod
   def calculate_health_score(cls, ship: Ship) -> float:
       max_health = {
           ShipType.LARGE: 3,
           ShipType.MEDIUM: 2,
           ShipType.SMALL: 1
       }
       current_health = max_health[ship.type] - ship.hit_count
       return (1 - current_health/max_health[ship.type]) * 9 + 3  # 3-9分

   @classmethod
   def calculate_firepower_score(cls, ship: Ship) -> float:
       total_missiles = ship.sonic_missiles + ship.hypersonic_missiles

       if ship.type == ShipType.LARGE:
           if total_missiles > 6: return 9.0
           elif total_missiles >= 4: return 6.0
           return 3.0
       elif ship.type == ShipType.MEDIUM:
           if total_missiles > 4: return 9.0
           elif total_missiles >= 2: return 6.0
           return 3.0
       else:  # SMALL
           if total_missiles == 3: return 9.0
           elif total_missiles == 2: return 6.0
           return 3.0

   @classmethod
   def evaluate_ship(cls, ship: Ship, enemy_ships: List[Ship]) -> DefenseShipScore:
       scores = {
           "threat": cls.calculate_threat_score(ship, enemy_ships),
           "health": cls.calculate_health_score(ship),
           "firepower": cls.calculate_firepower_score(ship)
       }

       weighted_total = (
           scores["threat"] * cls.THREAT_WEIGHT +
           scores["health"] * cls.HEALTH_WEIGHT +
           scores["firepower"] * cls.FIREPOWER_WEIGHT
       )

       return DefenseShipScore(scores, weighted_total)

class MovePositionMatrix:
   ESCAPE_WEIGHT = 0.6
   DISPERSION_WEIGHT = 0.4

   @classmethod
   def calculate_escape_score(cls, position: Tuple[int, int],
                            enemy_ships: List[Ship]) -> float:
       threats = sum(1 for enemy in enemy_ships
                    if cls.is_in_enemy_range(position, enemy))

       if threats == 0:
           return 9.0  # 完全離開威脅
       elif threats == 1:
           return 6.0  # 部分離開威脅
       return 3.0     # 仍在威脅中

   @staticmethod
   def is_in_enemy_range(position: Tuple[int, int], enemy: Ship) -> bool:
       dx = abs(position[0] - enemy.position[0])
       dy = abs(position[1] - enemy.position[1])
       distance = math.sqrt(dx*dx + dy*dy)
       return distance <= 5

   @classmethod
   def calculate_dispersion_score(cls, position: Tuple[int, int],
                                friendly_ships: List[Ship]) -> float:
       if not friendly_ships:
           return 9.0

       distances = []
       for friend in friendly_ships:
           dx = abs(position[0] - friend.position[0])
           dy = abs(position[1] - friend.position[1])
           distance = math.sqrt(dx*dx + dy*dy)
           distances.append(distance)

       avg_distance = sum(distances) / len(distances)

       if avg_distance > 4:
           return 9.0
       elif avg_distance >= 3:
           return 6.0
       return 3.0

   @classmethod
   def evaluate_position(cls, position: Tuple[int, int],
                        enemy_ships: List[Ship],
                        friendly_ships: List[Ship]) -> MovePositionScore:
       scores = {
           "escape": cls.calculate_escape_score(position, enemy_ships),
           "dispersion": cls.calculate_dispersion_score(position, friendly_ships)
       }

       weighted_total = (
           scores["escape"] * cls.ESCAPE_WEIGHT +
           scores["dispersion"] * cls.DISPERSION_WEIGHT
       )

       return MovePositionScore(scores, weighted_total)

   @classmethod
   def get_valid_positions(cls, ship: Ship, grid_size: int,
                         all_ships: List[Ship]) -> List[Tuple[int, int]]:
       valid_positions = []
       current_x, current_y = ship.position
       speed = ship.speed

       for dx in range(-speed, speed + 1):
           for dy in range(-speed, speed + 1):
               if abs(dx) + abs(dy) > speed:
                   continue

               new_x = current_x + dx
               new_y = current_y + dy

               if 0 <= new_x < grid_size and 0 <= new_y < grid_size:
                   position_occupied = any(
                       other.position == (new_x, new_y)
                       for other in all_ships
                       if other != ship
                   )

                   if not position_occupied:
                       valid_positions.append((new_x, new_y))

       return valid_positions

# 模擬及主程式

攻擊流程(_execute_combat_phase)包含： \\
選擇攻擊船艦 => 選擇攻擊目標 => 選擇飛彈

防禦流程(_execute_defense)包含： \\
選擇需要移動的船艦 => 選擇移動位置

戰鬥記錄系統：
* 攻擊方船艦資訊
* 目標船艦資訊
* 使用的飛彈類型
* 命中結果
* 覆蓋區域
* 移動的船艦資訊
* 原始位置
* 新位置
* 移動評分數據



In [ ]:
class AttackSystem:
    def __init__(self, missile_system: MissileSystem,
                 movement_system: MovementSystem,
                 grid_system: GridSystem,
                 battle_log_system: BattleLogSystem):

        self.missile_system = missile_system
        self.movement_system = movement_system
        self.grid_system = grid_system
        self.battle_log_system = battle_log_system


    def find_attack_position(self, attacker: Ship, target: Ship) -> Optional[Tuple[int, int]]:

        valid_positions = self.movement_system.get_valid_positions(
            attacker,
            self.grid_system.size,
            [ship for x in range(self.grid_system.size)
                 for y in range(self.grid_system.size)
                 if (ship := self.grid_system.get_ship_at_position((x, y))) is not None]
        )

        best_position = None
        min_distance = float('inf')

        for pos in valid_positions:
            distance = self.missile_system.calculate_distance(pos, target.position)

            sonic_in_range = (
                distance <= 4 and  # 音速飛彈射程
                (pos[0] == target.position[0] or pos[1] == target.position[1])  # 十字範圍
            )
            hypersonic_in_range = (
                distance <= 5 and  # 超音速飛彈射程
                abs(pos[0] - target.position[0]) <= 1 and
                abs(pos[1] - target.position[1]) <= 1  # 九宮格範圍
            )

            if (sonic_in_range or hypersonic_in_range) and distance < min_distance:
                min_distance = distance
                best_position = pos

        return best_position



    def move_to_attack_position(self, attacker: Ship, target: Ship,
                              round_num: int, is_first_attack: bool) -> bool:
        attack_position = self.find_attack_position(attacker, target)

        if attack_position is None:
            self.battle_log_system.log_attack_failed(
                round_num,
                is_first_attack,
                attacker,
                target,
                "no_valid_attack_position"
            )
            return False

        if attack_position == attacker.position:
            return True

        old_position = attacker.position
        self.grid_system.remove_ship(old_position)
        move_success = self.grid_system.place_ship(attacker, attack_position)

        if move_success:
            self.battle_log_system.log_attack_move(
                round_num,
                is_first_attack,
                attacker,
                old_position,
                attack_position,
                {"reason": "positioning_for_attack"}
            )

        return move_success




    def select_target(self, attacker: Ship, defenders: List[Ship],
                     target_matrix: TargetSelectionMatrix) -> Optional[Tuple[Ship, ShipScore]]:
        if not defenders:
            return None

        target_scores = [(defender, target_matrix.evaluate(defender))
                        for defender in defenders]
        return max(target_scores, key=lambda x: x[1].weighted_total)

    def select_missile(self, attacker: Ship, target: Ship,
                      missile_matrix: MissileSelectionMatrix) -> Optional[Tuple[MissileType, MissileScore]]:
        distance = self.missile_system.calculate_distance(attacker.position, target.position)

        missile_scores = [
            (missile_type, missile_matrix.evaluate(
                attacker,
                missile_type,
                distance,
                target.position
            ))
            for missile_type in MissileType
        ]

        return max(missile_scores, key=lambda x: x[1].weighted_total)


    def execute_attack(self, attacker: Ship, target: Ship,
                      missile_type: MissileType, round_num: int,
                      is_first_attack: bool) -> bool:

        if not self.move_to_attack_position(attacker, target, round_num, is_first_attack):
            return False

        if missile_type == MissileType.SONIC:
            if attacker.sonic_missiles <= 0:
                self.battle_log_system.log_attack_failed(
                    round_num, is_first_attack, attacker, target, "no_sonic_missiles")
                return False
            attacker.sonic_missiles -= 1
        else:  # HYPERSONIC
            if attacker.hypersonic_missiles <= 0:
                self.battle_log_system.log_attack_failed(
                    round_num, is_first_attack, attacker, target, "no_hypersonic_missiles")
                return False
            attacker.hypersonic_missiles -= 1

        hit_success = self.missile_system.check_hit(
            attacker.position, target.position, missile_type)

        covered_positions = self.missile_system.calculate_covered_positions(
            attacker.position, missile_type)

        was_sunk = False
        if hit_success:
            target.hit_count += 1
            was_sunk = target.is_sunk()

        self.battle_log_system.log_attack_result(
            round_num, is_first_attack, attacker, target,
            missile_type, hit_success, was_sunk, covered_positions)

        return hit_success


class DefenseSystem:
    def __init__(self, movement_system: MovementSystem,
                 grid_system: GridSystem,
                 battle_log_system: BattleLogSystem):
        self.movement_system = movement_system
        self.grid_system = grid_system
        self.battle_log_system = battle_log_system

    def select_ship_to_move(self, defending_ships: List[Ship],
                          enemy_ships: List[Ship],
                          defense_matrix: DefenseShipSelectionMatrix,
                          round_num: int,
                          is_first_attack: bool) -> Optional[Tuple[Ship, ShipScore]]:

        if not defending_ships or not enemy_ships:
            self.battle_log_system.log_defense_skip(
                round_num,
                is_first_attack,
                "no_valid_ships"
            )
            return None

        defense_scores = []
        for ship in defending_ships:
            score = defense_matrix.evaluate_ship(ship, enemy_ships)
            defense_scores.append((ship, score))

            self.battle_log_system.log_defense_evaluation(
                round_num,
                is_first_attack,
                ship,
                score.scores
            )

        return max(defense_scores, key=lambda x: x[1].weighted_total)

    def select_move_position(self, ship: Ship,
                           defending_ships: List[Ship],
                           enemy_ships: List[Ship],
                           move_matrix: MovePositionMatrix) -> Optional[Tuple[Tuple[int, int], MovePositionScore]]:
        valid_positions = self.movement_system.get_valid_positions(
            ship,
            self.grid_system.size,
            defending_ships + enemy_ships
        )

        if not valid_positions:
            return None

        position_scores = [
            (pos, move_matrix.evaluate_position(pos, enemy_ships, defending_ships))
            for pos in valid_positions
        ]

        return max(position_scores, key=lambda x: x[1].weighted_total)

    def execute_defense_move(self, ship: Ship, new_position: Tuple[int, int],
                           round_num: int, is_first_attack: bool,
                           move_score: MovePositionScore) -> bool:

        if not ship.has_appeared or ship.is_sunk():
            self.battle_log_system.log_defense_skip(
                round_num,
                is_first_attack,
                "ship_not_available"
            )
            return False

        if not self.movement_system.validate_movement(
            ship,
            new_position,
            self.grid_system.grid
        ):
            self.battle_log_system.log_defense_skip(
                round_num,
                is_first_attack,
                "invalid_movement"
            )
            return False

        old_position = ship.position

        self.grid_system.remove_ship(old_position)
        move_success = self.grid_system.place_ship(ship, new_position)

        if move_success:
            self.battle_log_system.log_defense_move(
                round_num,
                is_first_attack,
                ship,
                old_position,
                new_position,
                move_score.scores
            )

        return move_success



class NavalBattleSimulation(NavalBattleBase):
    def __init__(self, grid_size: int = 8):

        super().__init__(grid_size)

        self.battle_log_system = BattleLogSystem()
        self.missile_system = MissileSystem(grid_size)
        self.movement_system = MovementSystem(grid_size)
        self.grid_system = GridSystem(grid_size)  # 確保有 grid_system

        self.fleet_system = FleetSystem(self.grid_system)

        self.defense_system = DefenseSystem(
            self.movement_system,
            self.grid_system,
            self.battle_log_system
        )

        self.attack_system = AttackSystem(
            missile_system=self.missile_system,
            movement_system=self.movement_system,
            grid_system=self.grid_system,
            battle_log_system=self.battle_log_system
        )

        self.attacker_matrix = AttackerSelectionMatrix()
        self.target_matrix = TargetSelectionMatrix()
        self.missile_matrix = MissileSelectionMatrix()
        self.defense_matrix = DefenseShipSelectionMatrix()
        self.move_position_matrix = MovePositionMatrix()

        self.performance_metrics = {
            'win_rate': 0.0,
            'survival_rate': 0.0,
            'destruction_rate': 0.0
        }

        self.first_attacker = random.choice(['A', 'B'])

        self.team_A = self.fleet_system.team_A
        self.team_B = self.fleet_system.team_B

        self.battle_log_system.log_game_start({
            'first_attacker': self.first_attacker,
            'grid_size': grid_size,
            'team_A_ships': self._count_ships(self.team_A),
            'team_B_ships': self._count_ships(self.team_B)
        })

    def _count_ships(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, int]:
        return {
            'large': len(team[ShipType.LARGE]),
            'medium': len(team[ShipType.MEDIUM]),
            'small': len(team[ShipType.SMALL])
        }

    def _count_alive_ships(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, int]:
        alive_counts = {'large': 0, 'medium': 0, 'small': 0}
        for ship_type, ships in team.items():
            for ship in ships:
                if not ship.is_sunk():
                    alive_counts[ship_type.value] += 1
        return alive_counts

    def calculate_survival_rate(self, team: Dict[ShipType, List[Ship]]) -> Dict[str, float]:
        survival_rates = {}
        for ship_type, ships in team.items():
            total = len(ships)
            alive = sum(1 for ship in ships if not ship.is_sunk())
            survival_rates[ship_type.value] = alive / total if total > 0 else 0
        return survival_rates

    def _check_winner(self) -> Optional[str]:
        team_a_ships = self.fleet_system.get_active_ships(self.team_A)
        team_b_ships = self.fleet_system.get_active_ships(self.team_B)

        if not team_a_ships:
            return 'B'
        elif not team_b_ships:
            return 'A'

        team_a_survival = self.calculate_survival_rate(self.team_A)
        team_b_survival = self.calculate_survival_rate(self.team_B)

        team_a_total = sum(team_a_survival.values())
        team_b_total = sum(team_b_survival.values())

        if team_a_total > team_b_total * 1.5:
            return 'A'
        elif team_b_total > team_a_total * 1.5:
            return 'B'

        return None

    def get_winner(self) -> Optional[str]:
        return self._check_winner()

    def simulate_battle(self, rounds: int = 3) -> Dict:
        battle_results = {
            'team_A': {'survival_rates': [], 'destruction_rates': []},
            'team_B': {'survival_rates': [], 'destruction_rates': []}
        }

        for round_num in range(rounds):
            self.advance_round()
            self._simulate_round(round_num + 1, battle_results)

            winner = self._check_winner()
            if winner:
                battle_results['winner'] = winner
                battle_results['end_round'] = round_num + 1
                break

        if 'winner' not in battle_results:
            team_a_survival = sum(self.calculate_survival_rate(self.team_A).values())
            team_b_survival = sum(self.calculate_survival_rate(self.team_B).values())

            if team_a_survival > team_b_survival:
                battle_results['winner'] = 'A'
            elif team_b_survival > team_a_survival:
                battle_results['winner'] = 'B'
            else:
                battle_results['winner'] = 'draw'

            battle_results['end_round'] = rounds

        self.performance_metrics['win_rate'] = self._calculate_win_rate(battle_results)
        self.performance_metrics['survival_rate'] = self._calculate_survival_rate(battle_results)
        self.performance_metrics['destruction_rate'] = self._calculate_destruction_rate(battle_results)

        return {**battle_results, **self.performance_metrics}


    def _simulate_round(self, round_num: int, battle_results: Dict) -> None:
        self.current_round = round_num

        first_team, second_team = self._get_round_teams()

        self._execute_combat_phase(first_team, second_team, is_first_attack=True)

        self._execute_combat_phase(second_team, first_team, is_first_attack=False)

        self._record_round_results(battle_results)

    def _execute_combat_phase(
        self,
        attacking_team: Dict[ShipType, List[Ship]],
        defending_team: Dict[ShipType, List[Ship]],
        is_first_attack: bool
    ) -> None:
        active_attackers = self.fleet_system.get_active_ships(attacking_team)
        active_defenders = self.fleet_system.get_active_ships(defending_team)

        if not active_attackers or not active_defenders:
            return

        target_result = self.attack_system.select_target(
            active_attackers[0],  # 選擇第一艘船作為攻擊者
            active_defenders,
            self.target_matrix
        )

        if target_result:
            target_ship, target_score = target_result

            missile_result = self.attack_system.select_missile(
                active_attackers[0],
                target_ship,
                self.missile_matrix
            )

            if missile_result:
                missile_type, missile_score = missile_result
                self.attack_system.execute_attack(
                    active_attackers[0],
                    target_ship,
                    missile_type,
                    self.current_round,
                    is_first_attack
                )

        self._execute_defense(defending_team, attacking_team, is_first_attack)

    def _execute_defense(
        self,
        defending_team: Dict[ShipType, List[Ship]],
        enemy_team: Dict[ShipType, List[Ship]],
        is_first_attack: bool
    ) -> None:
        active_defenders = self.fleet_system.get_active_ships(defending_team)
        active_enemies = self.fleet_system.get_active_ships(enemy_team)

        if not active_defenders or not active_enemies:
            return

        ship_result = self.defense_system.select_ship_to_move(
            active_defenders,
            active_enemies,
            self.defense_matrix,
            self.current_round,
            is_first_attack
        )

        if ship_result:
            ship_to_move, defense_score = ship_result

            position_result = self.defense_system.select_move_position(
                ship_to_move,
                active_defenders,
                active_enemies,
                self.move_position_matrix
            )

            if position_result:
                new_position, move_score = position_result
                self.defense_system.execute_defense_move(
                    ship_to_move,
                    new_position,
                    self.current_round,
                    is_first_attack,
                    move_score
                )


    def _get_round_teams(self) -> Tuple[Dict[ShipType, List[Ship]], Dict[ShipType, List[Ship]]]:
        if self.first_attacker == 'A':
            return self.fleet_system.team_A, self.fleet_system.team_B
        return self.fleet_system.team_B, self.fleet_system.team_A

    def _record_round_results(self, battle_results: Dict) -> None:
        for team_name in ['A', 'B']:
            team = self.fleet_system.team_A if team_name == 'A' else self.fleet_system.team_B
            survival_rate = self.calculate_survival_rate(team)
            battle_results[f'team_{team_name}']['survival_rates'].append(survival_rate)

    def get_battle_statistics(self) -> Dict:
        stats = {
            'rounds_played': self.current_round,
            'first_attacker': self.first_attacker,
            'team_A': {
                'initial_ships': self._count_ships(self.team_A),
                'remaining_ships': self._count_alive_ships(self.team_A),
                'total_attacks': 0,
                'successful_hits': 0,
                'ships_lost': 0
            },
            'team_B': {
                'initial_ships': self._count_ships(self.team_B),
                'remaining_ships': self._count_alive_ships(self.team_B),
                'total_attacks': 0,
                'successful_hits': 0,
                'ships_lost': 0
            }
        }

        battle_logs = self.battle_log_system.get_battle_log()

        for log in battle_logs:
            if log['type'] == 'attack_result':
                attacking_team = 'team_A' if 'Team_A' in log['attacker'] else 'team_B'
                stats[attacking_team]['total_attacks'] += 1
                if log['hit']:
                    stats[attacking_team]['successful_hits'] += 1
                if log['target_sunk']:
                    defending_team = 'team_B' if attacking_team == 'team_A' else 'team_A'
                    stats[defending_team]['ships_lost'] += 1

        return stats

    def _calculate_win_rate(self, results: Dict) -> float:
        return 1.0 if results.get('winner') == 'A' else 0.0

    def _calculate_survival_rate(self, results: Dict) -> float:
        survival_rates = results['team_A']['survival_rates'][-1]
        return sum(survival_rates.values()) / len(survival_rates)

    def _calculate_destruction_rate(self, results: Dict) -> float:
        survival_rates = results['team_A']['survival_rates'][-1]

        return 1 - sum(survival_rates.values()) / len(survival_rates)

    def reset_simulation(self):
        self.__init__(self.grid_size)





    def print_battle_report(self) -> None:
        stats = self.get_battle_statistics()
        winner = self._check_winner()

        print("\n=== 戰鬥報告 ===")
        print(f"總回合數: {stats['rounds_played']}")
        print(f"先手隊伍: Team {stats['first_attacker']}")
        if winner:
            print(f"勝利隊伍: Team {winner}")

        for team in ['A', 'B']:
            team_stats = stats[f'team_{team}']
            print(f"\n=== Team {team} 統計 ===")
            print("初始船艦數量:")
            for ship_type, count in team_stats['initial_ships'].items():
                print(f"  {ship_type}: {count}")
            print("剩餘船艦數量:")
            for ship_type, count in team_stats['remaining_ships'].items():
                print(f"  {ship_type}: {count}")
            print(f"總攻擊次數: {team_stats['total_attacks']}")
            print(f"成功命中次數: {team_stats['successful_hits']}")
            hit_rate = (team_stats['successful_hits'] / team_stats['total_attacks'] * 100
                       if team_stats['total_attacks'] > 0 else 0)
            print(f"命中率: {hit_rate:.1f}%")
            print(f"損失船艦數: {team_stats['ships_lost']}")


    def export_battle_log(self, filename: str) -> None:
        import json

        battle_logs = self.battle_log_system.get_battle_log()
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(battle_logs, f, indent=2, ensure_ascii=False)







'''
主程式和模擬
'''
def main():
    """主程式入口"""
    # 設定模擬參數
    num_simulations = 5
    grid_size = 8
    rounds = 20
    all_results = []

    print("=== 海戰模擬開始 ===")
    print(f"模擬次數: {num_simulations}")
    print(f"網格大小: {grid_size}x{grid_size}")
    print(f"最大回合數: {rounds}\n")

    # 執行多次模擬
    for sim in range(num_simulations):
        print(f"\n執行第 {sim+1} 次模擬")
        game = NavalBattleSimulation(grid_size=grid_size)

        # 輸出初始狀態
        print(f"先手隊伍: Team {game.first_attacker}")
        print("\n初始船艦配置:")
        print("Team A:", game._count_ships(game.team_A))
        print("Team B:", game._count_ships(game.team_B))

        # 執行模擬
        results = game.simulate_battle(rounds=rounds)
        all_results.append(results)

        # 輸出詳細戰鬥報告
        game.print_battle_report()

        # 導出戰鬥記錄
        game.export_battle_log(f"battle_log_sim_{sim+1}.json")

    # 分析所有模擬結果
    print("\n=== 總體統計 ===")
    aggregated_stats = {
        'team_A_wins': 0,
        'team_B_wins': 0,
        'draws': 0,
        'avg_rounds': 0,
        'team_A_avg_survival': {
            'large': 0,
            'medium': 0,
            'small': 0
        },
        'team_B_avg_survival': {
            'large': 0,
            'medium': 0,
            'small': 0
        }
    }

    for sim_num, results in enumerate(all_results, 1):
        # 計算勝負
        final_a = sum(results['team_A']['survival_rates'][-1].values())
        final_b = sum(results['team_B']['survival_rates'][-1].values())

        if final_a > final_b:
            aggregated_stats['team_A_wins'] += 1
        elif final_b > final_a:
            aggregated_stats['team_B_wins'] += 1
        else:
            aggregated_stats['draws'] += 1

        # 計算平均回合數
        aggregated_stats['avg_rounds'] += len(results['team_A']['survival_rates'])

        # 計算平均存活率
        for ship_type in ['large', 'medium', 'small']:
            aggregated_stats['team_A_avg_survival'][ship_type] += (
                results['team_A']['survival_rates'][-1][ship_type]
            )
            aggregated_stats['team_B_avg_survival'][ship_type] += (
                results['team_B']['survival_rates'][-1][ship_type]
            )

    # 計算平均值
    aggregated_stats['avg_rounds'] /= num_simulations
    for team in ['team_A_avg_survival', 'team_B_avg_survival']:
        for ship_type in aggregated_stats[team]:
            aggregated_stats[team][ship_type] /= num_simulations

    # 輸出總體統計結果
    print(f"\nTeam A 勝率: {aggregated_stats['team_A_wins']/num_simulations*100:.1f}%")
    print(f"Team B 勝率: {aggregated_stats['team_B_wins']/num_simulations*100:.1f}%")
    print(f"平手率: {aggregated_stats['draws']/num_simulations*100:.1f}%")
    print(f"平均回合數: {aggregated_stats['avg_rounds']:.1f}")

    print("\nTeam A 平均存活率:")
    for ship_type, rate in aggregated_stats['team_A_avg_survival'].items():
        print(f"  {ship_type}: {rate*100:.1f}%")

    print("\nTeam B 平均存活率:")
    for ship_type, rate in aggregated_stats['team_B_avg_survival'].items():
        print(f"  {ship_type}: {rate*100:.1f}%")

    # 導出總體統計
    import json
    with open('simulation_stats.json', 'w', encoding='utf-8') as f:
        json.dump(aggregated_stats, f, indent=2)

    print("\n模擬完成! 詳細記錄已保存到JSON檔案中")

if __name__ == "__main__":
    main()


=== 海戰模擬開始 ===
模擬次數: 5
網格大小: 8x8
最大回合數: 20


執行第 1 次模擬
先手隊伍: Team A

初始船艦配置:
Team A: {'large': 4, 'medium': 2, 'small': 2}
Team B: {'large': 3, 'medium': 3, 'small': 3}
需要強制出現 1 艘隱藏船艦
Team_A_Large 成功出現在位置 (3, 5)
需要強制出現 3 艘隱藏船艦
Team_B_Large 成功出現在位置 (7, 2)
Team_B_Medium 成功出現在位置 (0, 2)
Team_B_Small 成功出現在位置 (5, 7)

=== 戰鬥報告 ===
總回合數: 20
先手隊伍: Team A

=== Team A 統計 ===
初始船艦數量:
  large: 4
  medium: 2
  small: 2
剩餘船艦數量:
  large: 3
  medium: 2
  small: 2
總攻擊次數: 6
成功命中次數: 6
命中率: 100.0%
損失船艦數: 1

=== Team B 統計 ===
初始船艦數量:
  large: 3
  medium: 3
  small: 3
剩餘船艦數量:
  large: 1
  medium: 3
  small: 3
總攻擊次數: 5
成功命中次數: 5
命中率: 100.0%
損失船艦數: 2

執行第 2 次模擬
先手隊伍: Team B

初始船艦配置:
Team A: {'large': 4, 'medium': 2, 'small': 2}
Team B: {'large': 3, 'medium': 3, 'small': 3}
需要強制出現 4 艘隱藏船艦
Team_A_Large 成功出現在位置 (3, 0)
Team_A_Large 成功出現在位置 (4, 7)
Team_A_Medium 成功出現在位置 (5, 1)
Team_A_Small 成功出現在位置 (2, 5)
需要強制出現 4 艘隱藏船艦
Team_B_Large 成功出現在位置 (4, 5)
Team_B_Medium 成功出現在位置 (7, 2)
Team_B_Small 成功出現在位置 (6, 6)
Team_B_Small 

# 敏感度分析及賽局分析
進階技術：敏感度分析
在實際應用中，權重可能會根據不同決策者的觀點發生改變。為了確保決策的穩定性，我們可以進行敏感度分析，測試權重的變化如何影響
最終結果。  <br>
假設調整：  <br>
• 準則1的權重變為 0.3  
• 準則2的權重變為 0.4
• 準則3的權重保持 0.3 <br>
這些變化可能是因為決策者認為準則2（例如，客戶滿意度）比其他準則更重要。  <br>
新的計算過程  <br>
• 選項 A：總分=(6×0.3)+(7×0.4)+(8×0.3)=1.8+2.8+2.4=7.0   <br>
• 選項 B：總分=(5×0.3)+(9×0.4)+(6×0.3)=1.5+3.6+1.8=6.9   <br>
• 選項 C：總分=(8×0.3)+(6×0.4)+(7×0.3)=2.4+2.4+2.1=6.9  <br>
結果變化：新的評分顯示 選項 A 獲得了最高分，超過了之前的 選項 C，這顯示了權重調整對結果的影響

E. 決策優化 <br>
在攻防結束後，計算雙方剩餘的船艦數量。保留最多船艦的一方將獲得勝利。可使用
博弈論中的零和賽局模型來分析雙方的策略組合，並優化決策。 <br>
• 賽局矩陣  <br>
雙方的攻防策略組合可表示為賽局矩陣，其中每個策略組合對應不同的結果。通過計
算賽局矩陣中的最佳反應，來確定每一方的最優決策。

In [ ]:
#進行敏感度及賽局分析
class SensitivityAnalysis:
    """敏感度分析系統"""
    def __init__(self, simulation: NavalBattleSimulation):
        self.simulation = simulation
        self.results = {}
        self.weight_variations = {}
        self.base_performance = None

    def setup_weight_variations(self, matrix_type: str, step: float = 0.1):
        """設置權重變化範圍"""
        matrix = getattr(self.simulation, f"{matrix_type}_matrix")
        base_weights = matrix.get_weights()
        variations = {}

        for weight_key in base_weights:
            variations[weight_key] = []
            current = max(0.1, base_weights[weight_key] - 0.3)
            while current <= min(0.9, base_weights[weight_key] + 0.3):
                variations[weight_key].append(round(current, 2))
                current += step

        self.weight_variations[matrix_type] = variations
        print("Weight variations:", self.weight_variations)

    def run_analysis(self, rounds: int = 10):
        """執行敏感度分析"""
        # 記錄基準表現
        self.base_performance = self._evaluate_performance(rounds)

        for matrix_type, variations in self.weight_variations.items():
            matrix = getattr(self.simulation, f"{matrix_type}_matrix")
            base_weights = matrix.get_weights()

            for weight_key, values in variations.items():
                self.results[(matrix_type, weight_key)] = []

                for value in values:
                    self.simulation.reset_simulation()
                    # 調整權重
                    matrix.weight_config.adjust_weight(weight_key, value)
                    print(f"Current weights: {matrix.get_weights()}")  # 加入這行
                    # 評估表現
                    performance = self._evaluate_performance(rounds)
                    print(f"Performance with {weight_key}={value}: {performance}")  # 加入這行
                    # 記錄結果
                    self.results[(matrix_type, weight_key)].append({
                        'weight_value': value,
                        'performance': performance,
                        'change': self._calculate_change(performance)
                    })

                # 恢復原始權重
                matrix.set_weights(base_weights)

    def _evaluate_performance(self, rounds: int) -> Dict:
        """評估特定配置下的表現"""
        results = self.simulation.simulate_battle(rounds)
        print(f"Battle ended at round: {results.get('end_round')}")  # 加入這行
        return {
            'win_rate': results.get('win_rate', 0),
            'survival_rate': results.get('survival_rate', 0),
            'destruction_rate': results.get('destruction_rate', 0)
        }

    def _calculate_change(self, performance: Dict) -> Dict:
        """計算相對於基準表現的變化"""
        if not self.base_performance:
            return {}

        changes = {}
        for key, value in performance.items():
            base_value = self.base_performance[key]
            if base_value == 0:
                # 如果基準值為0，直接記錄絕對變化
                changes[key] = value * 100
            else:
                # 否則計算相對變化百分比
                changes[key] = ((value - base_value) / base_value * 100)

        return changes

    def analyze_results(self) -> Dict:
        """分析結果並生成報告"""
        analysis = {}

        for (matrix_type, weight_key), results in self.results.items():
            # 計算敏感度係數
            sensitivity = self._calculate_sensitivity(results)
            # 找出臨界點
            critical_points = self._find_critical_points(results)
            # 找出最優配置
            optimal_config = max(results, key=lambda x: x['performance']['win_rate'])

            analysis[(matrix_type, weight_key)] = {
                'sensitivity': sensitivity,
                'critical_points': critical_points,
                'optimal_value': optimal_config['weight_value'],
                'optimal_performance': optimal_config['performance']
            }

        return analysis

    def _calculate_sensitivity(self, results: List[Dict]) -> float:
        """計算敏感度係數"""
        if len(results) < 2:
            return 0.0

        changes = [r['change']['win_rate'] for r in results]
        weights = [r['weight_value'] for r in results]

        if len(changes) != len(weights):
            return 0.0

        # 使用平均變化率作為敏感度係數
        total_change = max(changes) - min(changes)
        total_weight_change = max(weights) - min(weights)

        return total_change / total_weight_change if total_weight_change != 0 else 0

    def _find_critical_points(self, results: List[Dict]) -> List[float]:
        """找出效果顯著變化的臨界點"""
        critical_points = []

        for i in range(1, len(results)):
            prev_performance = results[i-1]['performance']['win_rate']
            curr_performance = results[i]['performance']['win_rate']

            # 如果變化超過閾值，認為是臨界點
            if abs(curr_performance - prev_performance) > 0.1:
                critical_points.append(results[i]['weight_value'])

        return critical_points

    def generate_report(self) -> str:
        """生成分析報告"""
        analysis = self.analyze_results()
        report = ["=== 敏感度分析報告 ===\n"]

        for (matrix_type, weight_key), results in analysis.items():
            report.append(f"\n{matrix_type} - {weight_key}:")
            report.append(f"敏感度係數: {results['sensitivity']:.4f}")
            report.append(f"臨界點: {', '.join(map(str, results['critical_points']))}")
            report.append(f"最優值: {results['optimal_value']}")
            report.append("最優表現:")
            for metric, value in results['optimal_performance'].items():
                report.append(f"  {metric}: {value:.2f}")

        return "\n".join(report)


# 創建模擬實例
simulation = NavalBattleSimulation()

# 創建敏感度分析器
analyzer = SensitivityAnalysis(simulation)

# 設置要分析的權重變化
analyzer.setup_weight_variations('attacker')


# 運行分析
analyzer.run_analysis(rounds=50)

# 獲取分析報告
report = analyzer.generate_report()
print(report)


Weight variations: {'attacker': {'ammo': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], 'health': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 'mobility': [0.1, 0.2, 0.3, 0.4, 0.5]}}
需要強制出現 3 艘隱藏船艦
Team_A_Large 成功出現在位置 (7, 5)
Team_A_Medium 成功出現在位置 (0, 2)
Team_A_Small 成功出現在位置 (5, 0)
需要強制出現 1 艘隱藏船艦
Team_B_Medium 成功出現在位置 (5, 7)
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
目標不在攻擊範圍內
Battle ended at round: 50
Current weights: {'ammo': 0.1, 'health': 0.5, 'mobility': 0.4}
需要強制出現 5 艘隱藏船艦
Team_A_Large 成功出現在位置 (1, 5)
Team_A_Large 成功出現在位置 (6, 5)
Team_A_Large 成功出現在位置 (3, 3)
Team_A_Medium 成功出現在位置 (2, 0)
Team_A_Small 成功出現在位置 (2, 7)
需要強制出現 3 艘隱藏船艦
Team_B_Large 成功出現在位置 (2, 4)
Team_B_Medium 成功出現在位置 (4, 4)
Team_B_Small 成功出現在位置 (0, 3)
Battle ended at round: 50
Performance with ammo=0.1: {'win_rate': 1.0, 'survival_rate': 0.9166666666666666, 'destruction_rate': 0.08333333333333337}
Current weights: {'ammo': 0.2, 'health': 0.45, 'mobility': 0.35000000000000003}
需要

attacker - ammo:
敏感度係數: 0.0000
臨界點:
最優值: 0.1
最優表現:
  win_rate: 1.00
  survival_rate: 0.50
  destruction_rate: 0.50

attacker - health:
敏感度係數: 0.0000
臨界點:
最優值: 0.1
最優表現:
  win_rate: 1.00
  survival_rate: 0.50
  destruction_rate: 0.50

attacker - mobility:
敏感度係數: 0.0000
臨界點:
最優值: 0.1
最優表現:
  win_rate: 1.00
  survival_rate: 0.50
  destruction_rate: 0.50